In [1]:
import os 
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/benvictoria17/DataAnalytics/master/dataset/%E2%98%95Coffee%2C%20Rice%20and%20Beef%20Prices%20Changes%20for%2030%20Years/rice_beef_coffee_price_changes.csv")

In [3]:
data.shape

(360, 9)

In [4]:
data.head()

,Year,Month,Price_beef_kilo,Price_rice_kilo,Price_coffee_kilo,Inflation_rate,Price_rice_infl,Price_beef_infl,Price_coffee_infl
0,1992,Feb,2.52,0.28,1.51,89.59,0.53,4.78,2.86
1,1992,Mar,2.45,0.28,1.55,89.59,0.53,4.64,2.94
2,1992,Apr,2.38,0.28,1.45,89.59,0.53,4.51,2.75
3,1992,May,2.38,0.27,1.34,89.59,0.51,4.51,2.54
4,1992,Jun,2.33,0.27,1.30,89.59,0.51,4.42,2.46


In [5]:
data.isna().sum().to_frame("Number of missing values").rename_axis("Column Name")

,Number of missing values
Column Name,
Year,0
Month,0
Price_beef_kilo,0
Price_rice_kilo,0
Price_coffee_kilo,0
Inflation_rate,13
Price_rice_infl,13
Price_beef_infl,13
Price_coffee_infl,13


In [6]:
data[data["Inflation_rate"].isna()]

,Year,Month,Price_beef_kilo,Price_rice_kilo,Price_coffee_kilo,Inflation_rate,Price_rice_infl,Price_beef_infl,Price_coffee_infl
167,2006,Jan,2.62,0.29,2.74,NaN,NaN,NaN,NaN
168,2006,Feb,2.65,0.30,2.63,NaN,NaN,NaN,NaN
169,2006,Mar,2.62,0.30,2.51,NaN,NaN,NaN,NaN
170,2006,Apr,2.60,0.30,2.54,NaN,NaN,NaN,NaN
171,2006,May,2.62,0.31,2.41,NaN,NaN,NaN,NaN
172,2006,Jun,2.57,0.31,2.27,NaN,NaN,NaN,NaN
173,2006,Jul,2.62,0.32,2.31,NaN,NaN,NaN,NaN
174,2006,Aug,2.66,0.31,2.46,NaN,NaN,NaN,NaN
175,2006,Sep,2.68,0.31,2.42,NaN,NaN,NaN,NaN
176,2006,Oct,2.69,0.30,2.44,NaN,NaN,NaN,NaN


In [7]:
data.loc[data['Year'] == 2006, 'Inflation_rate'] = 31.96
data.loc[data['Year'] == 2022, 'Inflation_rate'] = 0.0

In [8]:
def adjust_for_inflation(price, inflation_rate):
    return round(price * (1 + inflation_rate/100), 2)
data.head(2)

,Year,Month,Price_beef_kilo,Price_rice_kilo,Price_coffee_kilo,Inflation_rate,Price_rice_infl,Price_beef_infl,Price_coffee_infl
0,1992,Feb,2.52,0.28,1.51,89.59,0.53,4.78,2.86
1,1992,Mar,2.45,0.28,1.55,89.59,0.53,4.64,2.94


In [9]:
print(f"$2.52 in 1992 with a cumulative inflation of 89.49% is worth ${adjust_for_inflation(2.52,89.59)} in 2022.")

$2.52 in 1992 with a cumulative inflation of 89.49% is worth $4.78 in 2022.


In [10]:
data.loc[data['Year'].isin([2006, 2022]), 'Price_rice_infl'] = data.loc[data['Year'].isin([2006, 2022])].apply(lambda row: adjust_for_inflation(row['Price_rice_kilo'], row['Inflation_rate']),axis=1)
data.loc[data['Year'].isin([2006, 2022]), 'Price_coffee_infl'] = data.loc[data['Year'].isin([2006, 2022])].apply(lambda row: adjust_for_inflation(row['Price_coffee_kilo'], row['Inflation_rate']),axis=1)
data.loc[data['Year'].isin([2006, 2022]), 'Price_beef_infl'] = data.loc[data['Year'].isin([2006, 2022])].apply(lambda row: adjust_for_inflation(row['Price_beef_kilo'], row['Inflation_rate']),axis=1)
data.isna().sum()

Year                 0
Month                0
Price_beef_kilo      0
Price_rice_kilo      0
Price_coffee_kilo    0
Inflation_rate       0
Price_rice_infl      0
Price_beef_infl      0
Price_coffee_infl    0
dtype: int64

In [11]:
data['Month&Year'] = data['Month'] + ' ' + data['Year'].astype(str)
data["Month&Year"].head(3)

0    Feb 1992
1    Mar 1992
2    Apr 1992
Name: Month&Year, dtype: object

In [12]:
import plotly.express as px
import plotly.graph_objects as go

melted_data = data.melt("Month&Year", value_vars=["Price_beef_infl", "Price_coffee_infl", "Price_rice_infl"],var_name="Price_commodity", value_name="Price (USD)")
melted_data.sample(5, random_state=123)

,Month&Year,Price_commodity,Price (USD)
968,Oct 2012,Price_rice_infl,0.65
209,Jul 2009,Price_beef_infl,3.46
790,Dec 1997,Price_rice_infl,0.45
1077,Nov 2021,Price_rice_infl,0.39
648,Feb 2016,Price_coffee_infl,3.61


In [13]:
line_coffee_hex = "#678ee8"
line_rice_hex = "#00d295"
line_beef_hex = "#ff875e"
fig = px.line(
    data_frame=melted_data,
    x="Month&Year",
    y="Price (USD)",
    color="Price_commodity",
    color_discrete_map = {"Price_beef_infl": line_beef_hex, "Price_coffee_infl": line_coffee_hex, "Price_rice_infl": line_rice_hex}
)

fig.update_layout(title_text='Commodity Prices 1992-2022 (Inflation Adjusted)',title_x=0.5)

fig.show()

In [14]:
fig = px.line(data_frame=data, x="Month&Year", y="Price_rice_infl",color_discrete_sequence=[line_rice_hex])
fig.add_vrect(x0="Sep 2007", x1="May 2008", line_width=0, fillcolor="green", opacity=0.2)
fig.add_vrect(x0="May 2008", x1="Dec 2008", line_width=0, fillcolor="red", opacity=0.2)
fig.update_layout(
    title="Rice Prices 1992-2022 (Inflation Adjusted)", 
    title_x = 0.5,
    xaxis=dict(title="Month&Year"), 
    yaxis=dict(title="Price (USD)"))
fig.show()

In [15]:
fig = px.line(data_frame=data, x="Month&Year", y="Price_beef_infl",color_discrete_sequence=[line_rice_hex])
fig.add_vrect(x0="Jan 2008", x1="Dec 2008", line_width=0, fillcolor="red", opacity=0.2) 
fig.add_vrect(x0="Jan 2014", x1="Dec 2015", line_width=0, fillcolor="blue", opacity=0.2) 
fig.add_vrect(x0="Dec 2019", x1=data.shape[0], line_width=0, fillcolor="violet", opacity=0.2) 
fig.update_layout(
    title="Beef Prices 1992-2022 (Inflation Adjusted)", 
    title_x = 0.5,
    xaxis=dict(title="Month&Year"), 
    yaxis=dict(title="Price (USD)"))
fig.show()

In [16]:
fig = px.line(data_frame=data, x="Month&Year", y="Price_coffee_infl",color_discrete_sequence=[line_coffee_hex])
fig.add_vrect(x0="Jan 1994", x1="Dec 1994", line_width=0, fillcolor="blue", opacity=0.2)
fig.add_vrect(x0="Jan 1997", x1="Dec 1997", line_width=0, fillcolor="violet", opacity=0.2)
fig.add_vrect(x0="Jun 2010", x1="Dec 2011", line_width=0, fillcolor="red", opacity=0.2)
fig.update_layout(
    title="Coffee Prices 1992-2022 (Inflation Adjusted)", 
    title_x = 0.5,
    xaxis=dict(title="Month&Year"), 
    yaxis=dict(title="Price (USD)"))
fig.show()

In [17]:
avg_prices_monthly = data.groupby('Month').mean().reset_index()
avg_prices_monthly

,Month,Year,Price_beef_kilo,Price_rice_kilo,Price_coffee_kilo,Inflation_rate,Price_rice_infl,Price_beef_infl,Price_coffee_infl
0,Apr,2006.5,3.132667,0.372000,2.983333,36.322667,0.482333,4.022333,3.921000
1,Aug,2006.5,3.197000,0.360000,3.008333,36.322667,0.468333,4.091667,3.961333
2,Dec,2006.5,3.193000,0.360333,3.007667,36.322667,0.470000,4.085000,3.942000
3,Feb,2006.5,3.066000,0.369000,2.968667,36.322667,0.483333,3.947333,3.904667
4,Jan,2007.5,3.172000,0.372667,3.048333,33.336333,0.479333,3.973333,3.912667
5,Jul,2006.5,3.155000,0.366000,2.992000,36.322667,0.477000,4.033333,3.949667
6,Jun,2006.5,3.121000,0.369000,2.959333,36.322667,0.479333,3.986333,3.902000
7,Mar,2006.5,3.109333,0.365667,2.990000,36.322667,0.476667,4.004000,3.935333
8,May,2006.5,3.145667,0.369000,3.042333,36.322667,0.478000,4.026667,4.024000
9,Nov,2006.5,3.221333,0.354000,3.004000,36.322667,0.461667,4.119667,3.937667


In [18]:
avg_prices_comm_infl = avg_prices_monthly.melt(id_vars=["Month"], value_vars=['Price_rice_infl', 'Price_beef_infl', 'Price_coffee_infl'],var_name="Price_commodity", value_name="Price (USD)")
avg_prices_comm_infl

,Month,Price_commodity,Price (USD)
0,Apr,Price_rice_infl,0.482333
1,Aug,Price_rice_infl,0.468333
2,Dec,Price_rice_infl,0.470000
3,Feb,Price_rice_infl,0.483333
4,Jan,Price_rice_infl,0.479333
5,Jul,Price_rice_infl,0.477000
6,Jun,Price_rice_infl,0.479333
7,Mar,Price_rice_infl,0.476667
8,May,Price_rice_infl,0.478000
9,Nov,Price_rice_infl,0.461667


In [19]:
fig = px.line(
    data_frame=avg_prices_comm_infl,
    x="Month",
    y="Price (USD)",
    color="Price_commodity",
    color_discrete_map = {"Price_beef_infl": line_beef_hex, "Price_coffee_infl": line_coffee_hex, "Price_rice_infl": line_rice_hex}
)

fig.update_layout(title_text='Commodity Prices Seasonal (Inflation Adjusted)',title_x=0.5)

fig.show()